In [2]:
# channels = ["WW","bb","tautau","mumu","nuenue","numunumu","nutaunutau"]
# channels = ["WW"]
channels = ["WW","numunumu"]

profile = ['NFW', 'Burkert']
masses = {"WW":[90, 8000], "bb":[15, 8000], 'tautau':[5, 4000], 'mumu':[5, 1000], "nuenue":[5, 200],"numunumu":[5, 200],"nutaunutau":[5,200]}

In [3]:

cfig_file = '/home/tchau/code/condor_submit/UL/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()

sub_file = '/home/tchau/code/condor_submit/UL/UL.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/UpperLimit_LLH-Interval.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 8GB'+'\n')
    s.write('getenv = True'+'\n')
    s.write('\n')
    s.write('arguments = -c $(c) -p $(p) -u $(u) -l $(l) -n $(n) --mc $(mc) -b $(bkg) --method $(method) --nsample $(nsample)'+'\n')

    s.write('queue')
    s.close()

In [4]:
import os
submit = '/home/tchau/code/condor_submit/UL/submit.sh'
n = 30
nsample = 200
bkgs=['FFT']
method = 'bisection'
# mcs=['0000', '1122']
# mcs = ['0000', '1122', '1124', '1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133']
mcs = ['1122']
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for bkg in bkgs:
        for mc in mcs:
            for c in channels:
                for p in profile:
                    path = '/home/tchau/code/condor_submit/UL/{}_{}_{}_{}_{}_{}_{}trial'.format(c, p, n, mc, bkg, method, nsample)
                    if not (os.path.exists(path)): os.makedirs(path)
                    dag_file = '{}/{}_{}_{}_{}_{}_{}_{}trial.dag'.format(path, c, p, n, bkg, mc, method, nsample)
                    with open(dag_file, 'w') as f:
                        f.write('JOB UL_{}_{}_{}_{}_{}_{}_{}trial.dag'.format(c,p, n, mc, bkg, method, nsample)+' UL.submit \n')
                        f.write('VARS UL_{}_{}_{}_{}_{}_{}_{}trial.dag'.format(c,p, n, mc, bkg, method, nsample)+' JOBNAME="UL_{}_{}_{}_{}_{}_{}_{}trial"'.format(c, p, n, mc, bkg, method, nsample) +' c="{}" p="{}" u="{}" l="{}" n="{}" mc="{}" bkg="{}" method="{}" nsample="{}"'.format(c, p, masses[c][1], masses[c][0], n, mc, bkg, method, nsample)+'\n')
                    f.close()
                    s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
                    print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/WW_NFW_30_1122_FFT_bisection_200trial/WW_NFW_30_FFT_1122_bisection_200trial.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/WW_Burkert_30_1122_FFT_bisection_200trial/WW_Burkert_30_FFT_1122_bisection_200trial.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/numunumu_NFW_30_1122_FFT_bisection_200trial/numunumu_NFW_30_FFT_1122_bisection_200trial.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/numunumu_Burkert_30_1122_FFT_bisection_200trial/numunumu_Burkert_30_FFT_1122_bisection_200trial.dag

